In [1]:
# Librerias necesarias para el ajuste del modelo SIR y el pronóstico del mismo 
import numpy as np
import pandas as pd
import os
import os.path
import datetime
from scipy.integrate import odeint
import matplotlib.pyplot as plt
import plotly.offline as py
!pip install cufflinks
import cufflinks as cf
!pip install plotly==4.9.0
import plotly.graph_objects as go
from sklearn import metrics
from sklearn.metrics import mean_squared_error

In [ ]:
# Creación de los DataFrames para Medellín

In [13]:
#DataFrame de Activos ciudad de Medellín
df = pd.read_csv(os.path.join('../Output', 'data_medellin.csv'))
df = df[['FECHA','ACTIVOS']]
df2 = pd.to_datetime(df['FECHA'])
df.index = df2
data_activos = df.drop(['FECHA'], axis=1)

,ACTIVOS
FECHA,
2020-03-09,1.0
2020-03-11,3.0
2020-03-14,5.0
2020-03-15,5.0
2020-03-19,8.0
...,...
2020-08-31,10989.0
2020-09-01,10399.0
2020-09-02,10220.0


In [4]:
# Se define las funciones para el modelo SIR y se plantea una solución por método de tanteo para hallar el beta y gamma
# que generen el menor error "mse" cuando se realiza la compraración entre el modelo y los datos reales.


In [5]:
# Rutina de auto-ajuste para el modelo SIR
def modelo_SIR(beta,gamma,t):
    N=100000
    I0, R0=1, 0
    S0=N - I0 - R0
    
# Las ecuaciones diferenciales del modelo SIR
    def deriv(y, t, N, beta, gamma):
        S, I, R = y
        dSdt = -beta * S * I / N
        dIdt = beta * S * I / N - gamma * I
        dRdt = gamma * I
        return dSdt, dIdt, dRdt

# Vector de las condiciones iniciales
    y0 = S0, I0, R0
    # Resolver el sistema de ecuaciones diferenciales, en la secuencia de días que ya definimos
    ret = odeint(deriv, y0, t, args=(N, beta, gamma))
    S, I, R = ret.T
    return (I)

# Se tomaron valores muy pequeños para abarcar todas las posibles combinaciones de los parámetros (beta y gamma)
beta_x = 0.01
gamma_x = 0.015
mse_x = 1000000000000

# datos historicos(reales)
real=data_activos['ACTIVOS']
t = np.linspace(0, len(data_activos), len(data_activos))

# Método de tanteo encontrando el beta y gamma óptimos que generan el menor "mse"
for beta in np.arange(0.01,0.3,0.01):
    for gamma in np.arange(0.015,0.3,0.005):
        I = modelo_SIR(beta,gamma,t)
        mse = mean_squared_error(real,I)
        if mse < mse_x:
            beta_x = beta
            gamma_x = gamma
            mse_x = mse
        print(beta_x,gamma_x,mse_x)

0.16 0.085 1079611.3830163095


In [6]:
# Modelo SIR con el beta y gamma óptimos encontrados en el paso anterior.
N =   100000
I0, R0 = 1, 0
S0 = N - I0 - R0
beta, gamma = beta_x, gamma_x

# Periodo de tiempo hasta la fecha de corte + 90 días
t1 = np.linspace(0, len(data_activos)+90, len(data_activos)+90)

# Ecuaciones diferenciales del SIR
def deriv(y, t1, N, beta, gamma):
    S, I, R = y
    dSdt = -beta * S * I / N
    dIdt = beta * S * I / N - gamma * I
    dRdt = gamma * I
    return dSdt, dIdt, dRdt

# Condiciones iniciales
y0 = S0, I0, R0

# Solución de las ecuaciones diferenciales
ret = odeint(deriv, y0, t1, args=(N, beta, gamma))
S, I, R = ret.T

In [17]:
# SE genera el rango para el pronóstico
fecha_serie = pd.Series.first_valid_index(data_activos)
rango_serie = pd.date_range(start=fecha_serie, periods=len(data_activos)+90, freq='d')

In [19]:
# Grafica del pronóstico SIR vs datos historicos(reales)
fig = go.Figure()
fig.add_trace(go.Scatter(x=rango_serie, y=S,mode='lines',name='Susceptible'))
fig.add_trace(go.Scatter(x=rango_serie, y=R,mode='lines',name='Recuperada y fallecidos'))
fig.add_trace(go.Scatter(x=rango_serie, y=I,mode='lines',name='Infectada'))
fig.add_trace(go.Scatter(x=rango_serie, y=data_activos['ACTIVOS'],mode='lines',name='Activos Reales'))
fig.update_layout(
        hovermode='x',
        font=dict(
            family="Courier New, monospace",
            size=14,
            color='#3F3F3F',
        ),
        legend=dict(
            x=0.02,
            y=1,
            traceorder="normal",
            font=dict(
                family="sans-serif",
                size=12,
                color= '#474747',
            ),
            bgcolor='#FFFFFF',
            borderwidth=3
        ),
        paper_bgcolor='#FFFFFF',
        plot_bgcolor='#F4EEEF',
        margin=dict(l=0, 
                    r=0, 
                    t=0, 
                    b=0
                    ),
      

    )
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='#D9D8D8')
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='#D9D8D8')

fig.update_yaxes(zeroline=True, zerolinewidth=2, zerolinecolor='#474747')

fig.show()

fig.write_html('../Templates/med_forecast_SIR.html')